In [ ]:
!pip install 'crewai[tools]'

In [ ]:
!pip install pandas openpyxl python-docx

In [ ]:
!pip install crewai
!pip install langchain
!pip install openai

In [2]:
import os
import openai

os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'
os.environ["OPENAI_API_KEY"] = your openai api key

In [5]:
import pandas as pd

file_path = '/content/mobile_forensic_data.xlsx'

try:
    # pandas를 이용해 엑셀 파일을 열고 첫 번째 시트를 읽음
    df = pd.read_excel(file_path, sheet_name=0)
    print(df.head())  # 첫 5개의 행을 출력하여 파일이 제대로 열리는지 확인
except FileNotFoundError:
    print("File not found. Please check the path and try again.")
except Exception as e:
    print(f"An error occurred: {e}")


  Device Name OS Version Serial Number             IMEI
0   iPhone 12   iOS 14.4        ABC123  123456789012345


In [36]:
import pandas as pd
from crewai import Agent, Task, Crew
from langchain.tools import StructuredTool
from datetime import datetime

class ExcelAnalyzer:
    def __init__(self, file_path):
        self.excel_file = pd.ExcelFile(file_path)

    def analyze_sheets(self) -> str:
        """Analyze each sheet in the Excel file"""
        analysis = {}
        for sheet_name in self.excel_file.sheet_names:
            df = pd.read_excel(self.excel_file, sheet_name=sheet_name)
            analysis[sheet_name] = {
                'rows': df.shape[0],
                'columns': df.shape[1],
                'column_names': df.columns.tolist(),
                'summary': df.describe().to_dict()
            }
        return str(analysis)

    def get_sheet_data(self, sheet_name: str) -> str:
        """Get data from a specific sheet"""
        if sheet_name not in self.excel_file.sheet_names:
            return f"Error: Sheet '{sheet_name}' not found. Available sheets are: {', '.join(self.excel_file.sheet_names)}"
        return str(pd.read_excel(self.excel_file, sheet_name=sheet_name).to_dict())

    def list_sheets(self) -> str:
        """List all sheet names in the Excel file"""
        return ", ".join(self.excel_file.sheet_names)

    def find_data_by_date(self, target_date: str) -> str:
        """
        Find data for a specific date across all sheets

        :param target_date: Date to search for in format 'YYYY.MM.DD'
        :return: String representation of found data
        """
        result = {}
        target_date = datetime.strptime(target_date, '%Y.%m.%d').date()

        for sheet_name in self.excel_file.sheet_names:
            df = pd.read_excel(self.excel_file, sheet_name=sheet_name)

            # Convert all date-like columns to datetime
            for col in df.columns:
                try:
                    df[col] = pd.to_datetime(df[col], errors='coerce').dt.date
                except Exception:
                    continue  # If conversion fails, skip this column

            # Filter rows where any date column matches the target date
            matching_rows = df[df.apply(lambda row: target_date in row.values, axis=1)]

            if not matching_rows.empty:
                result[sheet_name] = matching_rows.to_dict('records')

        if result:
            return str(result)
        else:
            return f"No data found for the date {target_date}"

class ReportWriter:
    def write_report(self, content: str) -> str:
        """Simply return the content as a string"""
        return content

# Initialize tools
excel_path = '/content/mobile_forensic_data.xlsx'  # Ensure this path is correct in your environment
excel_analyzer = ExcelAnalyzer(excel_path)
report_writer = ReportWriter()

# Define agents
excel_expert = Agent(
    role='Excel Analysis Expert',
    goal='Analyze the forensic Excel file thoroughly',
    backstory='You are an expert in data analysis with a focus on forensic data',
    tools=[
        StructuredTool.from_function(
            func=excel_analyzer.analyze_sheets,
            name="Analyze Excel Sheets",
            description="Analyze each sheet in the Excel file"
        ),
        StructuredTool.from_function(
            func=excel_analyzer.get_sheet_data,
            name="Get Sheet Data",
            description="Get data from a specific sheet. Input: sheet name"
        ),
        StructuredTool.from_function(
            func=excel_analyzer.list_sheets,
            name="List Sheets",
            description="List all sheet names in the Excel file"
        ),
        StructuredTool.from_function(
            func=excel_analyzer.find_data_by_date,
            name="Find Data by Date",
            description="Find data for a specific date across all sheets. Input: date in format 'YYYY.MM.DD'"
        )
    ],
    verbose=True
)

report_expert = Agent(
    role='Report Writing Expert',
    goal='Write a comprehensive report based on the forensic analysis',
    backstory='You are a skilled report writer specializing in forensic analysis reports',
    tools=[
        StructuredTool.from_function(
            func=report_writer.write_report,
            name="Write Report",
            description="Write content as a report string"
        )
    ],
    verbose=True
)

# Define tasks
task1 = Task(
    description='Analyze the forensic Excel file using pandas. First, list all sheets. Then, analyze each sheet and identify key information related to the incident on 2024.1.5',
    agent=excel_expert,
    expected_output="A detailed analysis of the Excel file, focusing on the incident from 2024.1.5"
)

task2 = Task(
    description='Write a report summarizing the analysis, focusing on the events of 2024.1.5. Include any relevant correlations found in the data.',
    agent=report_expert,
    expected_output="""
        Create a report and display it on the screen.
        Start with Device Info: Begin by providing information from the Device Info sheet.
        Focus on January 5, 2024 Incidents: Gather only the incidents that occurred on January 5, 2024, and include them in the report.
        Search by Date Across All Sheets: Examine each sheet in the Excel file, and if any information corresponds to the date January 5, 2024 include it in the report.
        Review All Sheets: After reviewing all sheets and including relevant data in the report, analyze and summarize the significance of the events that occurred on January 5, 2024.
        Conclusion: Provide a conclusion in the report that interprets the meaning of the events that took place on that date.
    """
)

# Create and run the crew
crew = Crew(
    agents=[excel_expert, report_expert],
    tasks=[task1, task2],
    verbose=0  # Increase verbosity for debugging
)

result = crew.kickoff()

# Print the content of the result
print("\n--- Forensic Analysis Report ---")
print(result)
print("\n--- End of Report ---")




> Entering new CrewAgentExecutor chain...
Action: List Sheets()
Action Input: {} 

Device Info, Account Info, WiFi Info, Call Logs, Messages, Photos

Thought: I have listed all the sheet names in the Excel file. Now, I will analyze each sheet to identify key information related to the incident on 2024.1.5.
Action: Analyze Excel Sheets()
Action Input: {} 

{'Device Info': {'rows': 1, 'columns': 4, 'column_names': ['Device Name', 'OS Version', 'Serial Number', 'IMEI'], 'summary': {'IMEI': {'count': 1.0, 'mean': 123456789012345.0, 'std': nan, 'min': 123456789012345.0, '25%': 123456789012345.0, '50%': 123456789012345.0, '75%': 123456789012345.0, 'max': 123456789012345.0}}}, 'Account Info': {'rows': 10, 'columns': 4, 'column_names': ['Account Type', 'Email', 'Username', 'Last Login'], 'summary': {'Account Type': {'count': 10, 'unique': 3, 'top': 'Google', 'freq': 4}, 'Email': {'count': 10, 'unique': 10, 'top': 'user1@gmail.com', 'freq': 1}, 'Username': {'count': 10, 'unique': 10, 'top': '